# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""30822320189374…","""87703364802919…","""../../../../da…",0.983033,"""48781323166019…","""51011248327500…",false
"""40418447798079…","""65679420397042…","""../../../../da…",0.144221,"""48781323166019…","""17946105668399…",true
"""48298920449231…","""97457444554673…","""../../../../da…",0.679745,"""85129578503858…","""51011248327500…",false
"""40418447798079…","""84177304240648…","""../../../../da…",1.0,"""10117297975099…","""51011248327500…",false
"""30822320189374…","""47264280020855…","""../../../../da…",0.718982,"""10117297975099…","""24691162197763…",true
"""48298920449231…","""83933306511350…","""../../../../da…",0.0,"""45457631020186…",null,false
"""48298920449231…","""90869651330505…","""../../../../da…",0.02333,"""10117297975099…","""24691162197763…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""32007213576534…","""43962117163167…","""../../../../da…",0.983033,"""FirstMortgage3…","""32145962838458…","""Rejected"""
"""17633045152570…","""82848587524843…","""../../../../da…",0.144221,"""FirstMortgage3…","""14336159632136…","""Accepted"""
"""12276676574179…","""13872867592253…","""../../../../da…",0.679745,"""MoneyMarketSav…","""32145962838458…","""Rejected"""
"""17633045152570…","""42788729166775…","""../../../../da…",1.0,"""BasicChecking""","""32145962838458…","""Rejected"""
"""32007213576534…","""16958947526539…","""../../../../da…",0.718982,"""BasicChecking""","""12614314789239…","""Accepted"""
"""12276676574179…","""36441061672860…","""../../../../da…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""12276676574179…","""16818932341553…","""../../../../da…",0.02333,"""BasicChecking""","""12614314789239…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""12239954207948…","""87252206029681…","""../../../../da…",0.983033,"""FirstMortgage3…","""61033389184269…","""Rejected"""
"""18266513357514…","""14482461766117…","""../../../../da…",0.144221,"""FirstMortgage3…","""76535242828531…","""Accepted"""
"""17997121913132…","""54658723223942…","""../../../../da…",0.679745,"""MoneyMarketSav…","""61033389184269…","""Rejected"""
"""18266513357514…","""18246302694057…","""../../../../da…",1.0,"""BasicChecking""","""61033389184269…","""Rejected"""
"""12239954207948…","""55972712658957…","""../../../../da…",0.718982,"""BasicChecking""","""33826803675116…","""Accepted"""
"""17997121913132…","""10648258141473…","""../../../../da…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""17997121913132…","""69107899863506…","""../../../../da…",0.02333,"""BasicChecking""","""33826803675116…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""79084140549082…","""53886862569516…",0.983033,"""FirstMortgage3…","""20171149883914…","""Rejected"""
"""12620251111267…","""13127443162985…",0.144221,"""FirstMortgage3…","""30916122936757…","""Accepted"""
"""15449059382387…","""48710983491231…",0.679745,"""MoneyMarketSav…","""20171149883914…","""Rejected"""
"""12620251111267…","""99180667342530…",1.0,"""BasicChecking""","""20171149883914…","""Rejected"""
"""79084140549082…","""31732814355852…",0.718982,"""BasicChecking""","""99172806354410…","""Accepted"""
"""15449059382387…","""18303492972203…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""15449059382387…","""30730729342891…",0.02333,"""BasicChecking""","""99172806354410…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""af486bc1815c72…","""c73023dda51de1…","""../../../../da…",0.983033,"""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""3cdf7b1ae1342e…","""5414b197fb87b3…","""../../../../da…",0.144221,"""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""806ef34eb8237f…","""892f5d61a9215e…","""../../../../da…",0.679745,"""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""3cdf7b1ae1342e…","""c3aa807d9e927a…","""../../../../da…",1.0,"""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""af486bc1815c72…","""c2d492f9a05073…","""../../../../da…",0.718982,"""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""806ef34eb8237f…","""b21059221605c4…","""../../../../da…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""806ef34eb8237f…","""80cbe489dcecd4…","""../../../../da…",0.02333,"""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
